In [1]:
import cv2 as cv
import numpy as np

from datetime import datetime as dt
def frame_stamp():
    current = dt.now()
    out_str = (str(current.year) + str(current.month) + str(current.day) + "_" + 
               str(current.hour) + str(current.minute) + str(current.second) + "_" +
               "frame")
    return out_str

In [2]:
move_frame = False
pts = []
def next_frame(event,x,y,flags,param):
    if event == cv.EVENT_LBUTTONDOWN:
        global pts
        pts = []
        pts.append(x)
        pts.append(y)

    elif event == cv.EVENT_LBUTTONUP:
        if (pts[0] < x) or (pts[1] < y):
            pts.append(x)
            pts.append(y)
            global move_frame
            move_frame = True

In [3]:
def classify(file = 'resources/test_video.mp4', width = 960, height = 540, flip = True):
    cap = cv.VideoCapture(file)
    ret, frame = cap.read()
    clone = frame.copy()
    clone = cv.resize(clone, (width,height))
    if(flip): clone = cv.flip(clone, 0)
    cv.namedWindow("Video")
    cv.setMouseCallback("Video", next_frame)
    global move_frame
    while ret:
        cv.imshow("Video", clone)
        key = cv.waitKey(1)
        
        skip = False
        if (key == 113):
            break
        if (key == 46):
            skip = True
        
        if (move_frame or skip):
            ret, clone = cap.read()
            clone = cv.resize(clone, (width,height))
            if(flip): clone = cv.flip(clone, 0)
            if (move_frame):
                move_frame = False
                roi = clone[pts[1]:pts[3], pts[0]:pts[2]]
                cv.imshow("ROI", roi)
                cv.imwrite("./labels/" + frame_stamp() + ".jpg", roi)
                cv.waitKey(0)
                cv.destroyWindow("ROI")

    cap.release()
    cv.destroyAllWindows()

In [4]:
def play_video(file = 'resources/test_video.mp4', width = 960, height = 540, flip = True):
    cap = cv.VideoCapture(file)
    ret, frame = cap.read()
    clone = cv.resize(frame, (width,height))
    if(flip): clone = cv.flip(clone, 0)
    while ret:
        cv.imshow("Input", clone)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        ret, frame = cap.read()
        clone = cv.resize(frame, (width,height))
        if(flip): clone = cv.flip(clone, 0)
    cap.release()
    cv.destroyAllWindows()

In [5]:
classify()

In [14]:
play_video()

In [ ]:
ssd = cv.readNetFromCaffe()